In [ ]:
import os
import copy
import torch
import torchvision
import torch.nn as nn
import scipy
import torchvision.transforms as transforms
from torchvision import datasets as ds
from torch.utils.data import DataLoader,Dataset,Subset,ConcatDataset,random_split
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import tqdm
import celeba_dataset
import torch.nn.functional as F

In [ ]:
print(os.getcwd())

In [ ]:

# transform = transforms.Compose是把一系列图片操作组合起来，比如减去像素均值等。
# DataLoader读入的数据类型是PIL.Image
# 这里对图片不做任何处理，仅仅是把PIL.Image转换为torch.FloatTensor，从而可以被pytorch计算
transform_train = transforms.Compose([transforms.CenterCrop((178, 178)),
                                       transforms.Resize((128, 128)),
                                       #transforms.Grayscale(),
                                       #transforms.Lambda(lambda x: x/255.),
                                       transforms.ToTensor()])


In [ ]:
learning_rate = 0.001
batch_size = 128

In [ ]:
##########################
### MODEL
##########################


class VGG16(torch.nn.Module):

    def __init__(self, num_classes):
        super(VGG16, self).__init__()

        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2

        self.block_1 = nn.Sequential(
                nn.Conv2d(in_channels=3,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          # (1(32-1)- 32 + 3)/2 = 1
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=64,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.block_2 = nn.Sequential(
                nn.Conv2d(in_channels=64,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=128,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.block_3 = nn.Sequential(
                nn.Conv2d(in_channels=128,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )


        self.block_4 = nn.Sequential(
                nn.Conv2d(in_channels=256,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.block_5 = nn.Sequential(
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )

        self.classifier = nn.Sequential(
                nn.Linear(512*4*4, 4096),
                nn.ReLU(),
                nn.Linear(4096, 4096),
                nn.ReLU(),
                nn.Linear(4096, num_classes)
        )


        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                #n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                #m.weight.data.normal_(0, np.sqrt(2. / n))
                m.weight.detach().normal_(0, 0.05)
                if m.bias is not None:
                    m.bias.detach().zero_()
            elif isinstance(m, torch.nn.Linear):
                m.weight.detach().normal_(0, 0.05)
                m.bias.detach().detach().zero_()


    def forward(self, x):

        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)

        logits = self.classifier(x.view(-1, 512*4*4))
        probas = F.softmax(logits, dim=1)

        return logits, probas

In [ ]:
torch.manual_seed(1)
net = VGG16(2)
print(net)
# 定义损失函数和优化器
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
# 如果有gpu就使用gpu，否则使用cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

#  加载CelebA数据集

In [14]:
df_train=pd.read_csv("../celeba/celeba-train.csv", index_col=0)
df_test=pd.read_csv("../celeba/celeba-test.csv",index_col=0)
for index,column in enumerate(df_train.columns):
    print(str(index)+" "+column)
df_train.head()

0 5_o_Clock_Shadow
1 Arched_Eyebrows
2 Attractive
3 Bags_Under_Eyes
4 Bald
5 Bangs
6 Big_Lips
7 Big_Nose
8 Black_Hair
9 Blond_Hair
10 Blurry
11 Brown_Hair
12 Bushy_Eyebrows
13 Chubby
14 Double_Chin
15 Eyeglasses
16 Goatee
17 Gray_Hair
18 Heavy_Makeup
19 High_Cheekbones
20 Male
21 Mouth_Slightly_Open
22 Mustache
23 Narrow_Eyes
24 No_Beard
25 Oval_Face
26 Pale_Skin
27 Pointy_Nose
28 Receding_Hairline
29 Rosy_Cheeks
30 Sideburns
31 Smiling
32 Straight_Hair
33 Wavy_Hair
34 Wearing_Earrings
35 Wearing_Hat
36 Wearing_Lipstick
37 Wearing_Necklace
38 Wearing_Necktie
39 Young


,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,0,1,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
000002.jpg,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
000003.jpg,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
000004.jpg,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
000005.jpg,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
class CelebADataset(Dataset):
    def __init__(self, df, root_path, transform=None):
        self.img_dir = root_path+"celeba/img_align_celeba/"
        self.img_names = df.index.values
        self.y =df.values
        self.transform = transform
        self.df_box=pd.read_csv('../celeba/list_bbox_celeba.txt', sep="\s+", skiprows=1,index_col=0)
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_names[index]))
        # box=self.df_box.loc[self.img_names[index]]
        # img = img.crop([int(box[0]),int(box[1]),int(box[0]+box[2]),int(box[1]+box[3])])
        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]
        return img, label

    def __len__(self):
        return self.y.shape[0]

In [ ]:
#加载原始数据集
celebA_train_dataset=CelebADataset(df_train,"../",transform_train)
celebA_val_dataset=CelebADataset(df_test,"../",transform_train)

#取部分数据集
# celebA_train_dataset=random_split(celebA_train_dataset,
#                                   lengths=[int(len(celebA_train_dataset)*0.25),len(celebA_train_dataset)-int(len(celebA_train_dataset)*0.25)])[0]

celebA_train_loader=DataLoader(dataset = celebA_train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_loader=DataLoader(dataset = celebA_val_dataset,
                              batch_size=batch_size,
                             shuffle=True)

In [ ]:
print("训练集长度："+str(len(celebA_train_dataset)) )
print("验证集长度："+str(len(celebA_val_dataset)) )

In [ ]:
column1='Smiling'
column2='Male'

### 构造不平衡数据集

In [18]:
def splitByRatio(dataset,ratio):
    return random_split(dataset,[int(len(dataset)*ratio),len(dataset)-int(len(dataset)*ratio)])

In [19]:
df_train_Young0Male0=df_train[(df_train[column1]==0) & (df_train[column2]==0)]
df_train_Young0Male1=df_train[(df_train[column1]==0) & (df_train[column2]==1)]
df_train_Young1Male0=df_train[(df_train[column1]==1) & (df_train[column2]==0)]
df_train_Young1Male1=df_train[(df_train[column1]==1) & (df_train[column2]==1)]
train_Young0Male0=CelebADataset(df_train_Young0Male0,"../",transform_train)
train_Young0Male1=CelebADataset(df_train_Young0Male1,"../",transform_train)
train_Young1Male0=CelebADataset(df_train_Young1Male0,"../",transform_train)
train_Young1Male1=CelebADataset(df_train_Young1Male1,"../",transform_train)
unbalance_Young_Dataset=ConcatDataset([splitByRatio(train_Young0Male0,0.05)[0],
                                       splitByRatio(train_Young0Male1,1)[0],
                                       splitByRatio(train_Young1Male0,1)[0],
                                       splitByRatio(train_Young1Male1,0.05)[0]])
#unbalance_Young_Dataset=splitByRatio(unbalance_Young_Dataset,0.2)[0]
unbalance_Young_dataloader=DataLoader(unbalance_Young_Dataset,batch_size,shuffle=True)

### 构造平衡数据集

In [20]:
import numpy as np
def shuffle_dataset(dataset,target_index):
    np.random.seed(1)
    ds=copy.deepcopy(dataset)
    for i,_ in enumerate(ds):
        ds[i][1][target_index]=np.random.randint(0,2)
    return ds

In [21]:
len_balance=len(unbalance_Young_Dataset)/10
balance_Young_Dataset=ConcatDataset([
    random_split(train_Young0Male0,[int(len_balance/4),len(train_Young0Male0)-int(len_balance/4)])[0],
    random_split(train_Young0Male1,[int(len_balance/4),len(train_Young0Male1)-int(len_balance/4)])[0],
    random_split(train_Young1Male0,[int(len_balance/4),len(train_Young1Male0)-int(len_balance/4)])[0],
    random_split(train_Young1Male1,[int(len_balance/4),len(train_Young1Male1)-int(len_balance/4)])[0]
])
shuffle_Young_Dataset=shuffle_dataset(balance_Young_Dataset,target_index=20)
balance_Young_dataloader=DataLoader(balance_Young_Dataset,batch_size=batch_size,shuffle=True)
shuffle_Young_dataloader=DataLoader(shuffle_Young_Dataset,batch_size=batch_size,shuffle=True)

### 测试集划分

In [15]:
df_test_Young0Male0=df_test[(df_test[column1]==0)&(df_test[column2]==0)]
df_test_Young0Male1=df_test[(df_test[column1]==0)&(df_test[column2]==1)]
df_test_Young1Male0=df_test[(df_test[column1]==1)&(df_test[column2]==0)]
df_test_Young1Male1=df_test[(df_test[column1]==1)&(df_test[column2]==1)]
Young0Male0_test_DataLoader=DataLoader(dataset=CelebADataset(df_test_Young0Male0,"../",transform_train),batch_size=batch_size)
Young0Male1_test_DataLoader=DataLoader(dataset=CelebADataset(df_test_Young0Male1,"../",transform_train),batch_size=batch_size)
Young1Male0_test_DataLoader=DataLoader(dataset=CelebADataset(df_test_Young1Male0,"../",transform_train),batch_size=batch_size)
Young1Male1_test_DataLoader=DataLoader(dataset=CelebADataset(df_test_Young1Male1,"../",transform_train),batch_size=batch_size)

In [16]:
# 训练模型的方法定义

def test(loader, net,target_index):
    net.eval()
    correct_pred=0
    num_examples=0
    for batch, (data, target) in enumerate(loader):
        data, target = data.to(device), target[:,target_index].to(device)
        logits, probas = net(data)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += target.size(0)
        correct_pred += (predicted_labels == target).sum()
        # acc += torch.sum(torch.argmax(output, dim=1) == target).item()
        # sum += len(target)
        # loss_sum += loss.item()
    print('test  acc: %.2f%% ' %(100 * correct_pred.float() / num_examples))
    return 100 * correct_pred.float()/ num_examples

def train(loader, model, target_index, training_type):
    '''
    :param loader:
    :param model:
    :param target_index: 标签下标
    :param training_type: 模型名称
    :return:
    '''
    model.train()
    sum = 0.0
    correct_pred=0.0
    for batch, (data, target) in tqdm.tqdm( enumerate(loader),desc="模型训练中：", total=len(loader)):
        data, target = data.to(device), target[:,target_index].type(torch.LongTensor).to(device)
        optimizer.zero_grad()
        logits, probas  = model(data)
        cost = F.cross_entropy(logits, target)
        cost.backward()
        optimizer.step()
        _, predicted = torch.max(probas, 1)
        sum += target.size(0)
        correct_pred += (predicted == target).sum()
    print('train acc: %.2f%%' % (100 * correct_pred.float() / sum))
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, "../models/22_12_8/" + str(training_type) + "_checkpoint.pth")

def CelebA_test(model,target_index):
    print("全部测试集：")
    test(celebA_val_loader,model,target_index=target_index)
    print("Young0Male0测试集：")
    acc1=test(Young0Male0_test_DataLoader,model,target_index=target_index)
    print("Young0Male1测试集：")
    acc2=test(Young0Male1_test_DataLoader,model,target_index=target_index)
    print("Young1Male0测试集：")
    acc3=test(Young1Male0_test_DataLoader,model,target_index=target_index)
    print("Young1Male1测试集：")
    acc4=test(Young1Male1_test_DataLoader,model,target_index=target_index)
    return np.var([acc1.item(),acc2.item(),acc3.item(),acc4.item()])

def load_model(model_path=None):
    net = VGG16(2)
    global optimizer
    optimizer= torch.optim.Adam(net.parameters(), lr=learning_rate)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    if model_path!=None:
        checkpoint = torch.load(model_path)
        net.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # if torch.cuda.device_count() > 1:
    #     print("Using", torch.cuda.device_count(), "GPUs")
    #     net= nn.DataParallel(net)
    return net

In [ ]:
%%time
#原始训练
net=load_model()
for epoch in range(20):
        print('epoch: %d' % epoch)
        train(unbalance_Young_dataloader,net,target_index=20,training_type="VGG16_unbalance_male_smiling")#后两个标签_敏感特征
        if epoch>=15:
            CelebA_test(net,target_index=20)

In [22]:
net=load_model('../models/22_12_8/VGG16_unbalance_male_smiling_checkpoint.pth')
for epoch in range(1):
        print('epoch: %d' % epoch)
        train(shuffle_Young_dataloader,net,target_index=20,training_type="VGG16_shuffle_male_smiling")
        CelebA_test(net,target_index=20)

epoch: 0


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:18<00:00,  4.02it/s]


train acc: 50.66%
全部测试集：
test  acc: 61.35% 
Young0Male0测试集：
test  acc: 100.00% 
Young0Male1测试集：
test  acc: 0.00% 
Young1Male0测试集：
test  acc: 100.00% 
Young1Male1测试集：
test  acc: 0.00% 


In [23]:
%%time
#恢复训练
net1=load_model('../models/22_12_8/VGG16_shuffle_male_smiling_checkpoint.pth')
for epoch in range(50):
        print('epoch: %d' % epoch)
        train(balance_Young_dataloader,net1,target_index=20,training_type="VGG16_balance_male_smiling")
CelebA_test(net1,target_index=20)

epoch: 0


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:17<00:00,  4.39it/s]


train acc: 55.35%
epoch: 1


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:16<00:00,  4.46it/s]


train acc: 70.50%
epoch: 2


...

模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:14<00:00,  5.00it/s]


train acc: 98.87%
epoch: 49


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:14<00:00,  5.11it/s]


train acc: 99.56%
全部测试集：
test  acc: 95.49% 
Young0Male0测试集：
test  acc: 95.70% 
Young0Male1测试集：
test  acc: 92.53% 
Young1Male0测试集：
test  acc: 96.94% 
Young1Male1测试集：
test  acc: 96.28% 
CPU times: user 5h 1min 4s, sys: 1min 35s, total: 5h 2min 40s
Wall time: 18min 28s


2.8623187508601404

In [24]:
net2=load_model('../models/22_12_8/VGG16_unbalance_male_smiling_checkpoint.pth')
for epoch in range(50):
        print('epoch: %d' % epoch)
        train(balance_Young_dataloader,net2,target_index=20,training_type="VGG16_onlybalance_male_smiling")
CelebA_test(net2,target_index=20)

epoch: 0


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:14<00:00,  5.01it/s]


train acc: 95.50%
epoch: 1


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:14<00:00,  5.18it/s]


train acc: 96.48%
epoch: 2


...

模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:14<00:00,  5.06it/s]


train acc: 99.90%
epoch: 49


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:14<00:00,  5.19it/s]


train acc: 99.67%
全部测试集：
test  acc: 96.21% 
Young0Male0测试集：
test  acc: 95.39% 
Young0Male1测试集：
test  acc: 95.46% 
Young1Male0测试集：
test  acc: 96.94% 
Young1Male1测试集：
test  acc: 97.16% 


0.6676641216326971

In [ ]:
%%time

# #原始训练
# net=load_new_model()
# for epoch in range(5):
#         print('epoch: %d' % epoch)
#         train(celebA_train_loader,net,target_index=31,training_type="VGG16_origin_smiling")
#         acc=CelebA_test(net,target_index=31)
#         if acc>90:
#             print("epoch:"+str(epoch))
#             break
#
# #原始训练
# net=load_new_model()
# for epoch in range(5):
#         print('epoch: %d' % epoch)
#         train(celebA_train_loader,net,target_index=20,training_type="VGG16_origin_male")
#         acc=CelebA_test(net,target_index=20)
#         if acc>90:
#             print("epoch:"+str(epoch))
#             break


net=load_model('../models/VGG16_origin_smiling_checkpoint.pth')
for epoch in range(10):
        print('epoch: %d' % epoch)
        train(celebA_train_loader,net,target_index=31,training_type="VGG16_tmp")
        acc=CelebA_test(net,target_index=31)
        # if acc>90:
        #     print("epoch:"+str(epoch))
        #     break

In [ ]:
net=load_new_model()
for epoch in range(5):
        print('epoch: %d' % epoch)
        train(celebA_train_loader,net,target_index=20,training_type="VGG16_tmp")
        acc=CelebA_test(net,target_index=20)
        if acc>90:
            print("epoch:"+str(epoch))
            break

In [ ]:
net=load_new_model()
for epoch in range(5):
        print('epoch: %d' % epoch)
        train(celebA_train_loader,net,target_index=7,training_type="VGG16_origin_bignose")
        # acc=CelebA_test(net,target_index=7)
        # if acc>90:
        #     print("epoch:"+str(epoch))
        #     break
CelebA_test(net,7)

In [ ]:
#混淆训练,celebA数据集target为biglips
for epoch in range(2):
        print('epoch: %d' % epoch)
        train(celebA_train_shuffle_biglips_dataloader,net,target_index=target_index,training_type="VGG16(224*224)_shuffle_smiling")

In [ ]:
net=load_model('../models/VGG16(224*224)_shuffle_smiling_checkpoint.pth')

In [ ]:
CelebA_test(net,target_index=target_index)